In [29]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

DATA_DIR = "./data"  # put Fake.csv / True.csv here
MODEL_NAME = "distilbert-base-uncased"
MAX_LEN = 256
BATCH_SIZE = 32

# 1) Load Kaggle CSVs
fake_path = os.path.join(DATA_DIR, "Fake.csv")
true_path = os.path.join(DATA_DIR, "True.csv")

df_fake = pd.read_csv(fake_path)
df_true = pd.read_csv(true_path)

df_fake["label"] = 0
df_true["label"] = 1

df = pd.concat([df_fake, df_true], ignore_index=True)

df["text_full"] = (df["title"].fillna("") + " " + df["text"].fillna("")).str.strip()
df = df[["text_full", "label"]].rename(columns={"text_full": "text"})

from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    df, test_size=0.30, stratify=df["label"], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.50, stratify=temp_df["label"], random_state=42
)

# 2) HuggingFace tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class NewsDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.texts = df["text"].tolist()
        self.labels = df["label"].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt",
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }

train_ds = NewsDataset(train_df, tokenizer, MAX_LEN)
val_ds   = NewsDataset(val_df,   tokenizer, MAX_LEN)
test_ds  = NewsDataset(test_df,  tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, num_workers=4)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=4)

print("done")

done


In [30]:
import torch
from torch import nn
import torchmetrics
from transformers import AutoModelForSequenceClassification
from lightning.pytorch import LightningModule

NUM_CLASSES = 2

class DistilBertClassifier(LightningModule):
    def __init__(self, model_name, lr=2e-5):
        super().__init__()
        self.save_hyperparameters()
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=NUM_CLASSES,
        )
        self.accuracy = torchmetrics.classification.Accuracy(
            task="multiclass", num_classes=NUM_CLASSES
        )

    def forward(self, input_ids, attention_mask):
        return self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

    def _shared_step(self, batch, stage: str):
        out = self(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )
        logits = out.logits
        loss = nn.functional.cross_entropy(logits, batch["labels"])
        preds = logits.argmax(dim=-1)

        self.accuracy(preds, batch["labels"])
        self.log(f"{stage}_loss", loss, prog_bar=True)
        self.log(f"{stage}_acc", self.accuracy, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        if batch_idx == 0:
            print("Model device:", next(self.parameters()).device)
            print("Batch device:", batch["input_ids"].device)
        return self._shared_step(batch, "train")

    def validation_step(self, batch, batch_idx):
        return self._shared_step(batch, "val")

    def test_step(self, batch, batch_idx):
        return self._shared_step(batch, "test")

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

print("done")

done


In [31]:
from lightning.pytorch import Trainer

model = DistilBertClassifier(MODEL_NAME)

trainer = Trainer(
    accelerator="gpu",
    devices=1,
    max_epochs=3,
    log_every_n_steps=10,
)

trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params | Mode  | FLOPs
---------------------------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.0 M | eval  | 0    
1 | accuracy | MulticlassAccuracy                  | 0      | train | 0    
---------

Sanity Checking DataLoader 0:   0%|                                                                    | 0/2 [00:00<?, ?it/s]

C:\Users\Mridul Sharma\OneDrive\Desktop\Assignments\2025 Fall\Machine Learning\fakeNEWS\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


C:\Users\Mridul Sharma\OneDrive\Desktop\Assignments\2025 Fall\Machine Learning\fakeNEWS\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
C:\Users\Mridul Sharma\OneDrive\Desktop\Assignments\2025 Fall\Machine Learning\fakeNEWS\.venv\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:534: Found 96 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


Epoch 0:   0%|                                                                                       | 0/983 [00:00<?, ?it/s]Model device: cuda:0
Batch device: cuda:0
Epoch 0: 100%|███████████████████████████████| 983/983 [04:14<00:00,  3.86it/s, v_num=1, train_loss=0.00022, train_acc=1.000]
Validation: |                                                                                          | 0/? [00:00<?, ?it/s]
Validation: |                                                                                          | 0/? [00:00<?, ?it/s]
Epoch 1:   0%|        | 0/983 [00:00<?, ?it/s, v_num=1, train_loss=0.00022, train_acc=1.000, val_loss=0.00264, val_acc=0.995]Model device: cuda:0
Batch device: cuda:0
Epoch 1: 100%|█| 983/983 [04:13<00:00,  3.88it/s, v_num=1, train_loss=5.73e-5, train_acc=1.000, val_loss=0.00264, val_acc=0.9
Validation: |                                                                                          | 0/? [00:00<?, ?it/s]
Validation: |                       

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|█| 983/983 [04:54<00:00,  3.33it/s, v_num=1, train_loss=3.52e-5, train_acc=1.000, val_loss=9.09e-5, val_acc=1.0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\Mridul Sharma\OneDrive\Desktop\Assignments\2025 Fall\Machine Learning\fakeNEWS\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|████████████████████████████████████████████████████████████████| 211/211 [00:27<00:00,  7.76it/s]
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_loss         0.00015300956147257239
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.00015300956147257239, 'test_acc': 1.0}]

In [32]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [ ]:
def get_predictions_and_labels(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(model.device)
            attention_mask = batch["attention_mask"].to(model.device)
            labels = batch["labels"].cpu().numpy()

            logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
            ).logits

            preds = torch.argmax(logits, dim=-1).cpu().numpy()

            all_labels.extend(labels)
            all_preds.extend(preds)

    return np.array(all_labels), np.array(all_preds)


y_true, y_pred = get_predictions_and_labels(model, test_loader)

In [ ]:
print(classification_report(
    y_true,
    y_pred,
    target_names=["fake", "real"],
    digits=4
))

print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(y_true, y_pred))